In [3]:
!pip install SQLAlchemy

In [29]:
import sqlalchemy
sqlalchemy.__version__

'1.3.1'

In [30]:
# Engine 생성
from sqlalchemy import create_engine

engine = create_engine("sqlite://", echo=True)
print(engine)

Engine(sqlite://)


In [31]:
type(engine)

sqlalchemy.engine.base.Engine

In [32]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

metadata = MetaData()
users = Table('users', metadata,
        Column('id', Integer, primary_key=True),
        Column('name', String),
        Column('fullname', String)
)

addresses = Table('addresses', metadata,
        Column('id', Integer, primary_key=True),
        Column('user_id', None, ForeignKey('users.id')),
        Column('email_address', String, nullable=False)
)

# metadata.create_all(engine)

In [33]:
# 수정 불가능한 immutable 상태로 들어가 있다.
metadata.tables

immutabledict({'users': Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None), 'addresses': Table('addresses', MetaData(bind=None), Column('id', Integer(), table=<addresses>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.id'), table=<addresses>), Column('email_address', String(), table=<addresses>, nullable=False), schema=None)})

In [34]:
metadata.bind

In [35]:
metadata.create_all(engine)

2019-07-09 11:46:42,206 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 11:46:42,207 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:46:42,208 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 11:46:42,209 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:46:42,210 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 11:46:42,210 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:46:42,211 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-07-09 11:46:42,212 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:46:42,213 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 11:46:42,213 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:46:42,214 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 11:46:42,215 INFO sqlalchemy.engine.b

In [36]:
# users 라는 테이블에 insert 라는 메소드 사용하여 insert 출력
insert = users.insert()
print(insert)

insert = users.insert().values(
    name = 'kim', fullname = 'Anonymous, Kim')
print(insert)

insert.compile().params

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)
INSERT INTO users (name, fullname) VALUES (:name, :fullname)


{'name': 'kim', 'fullname': 'Anonymous, Kim'}

In [37]:
conn = engine.connect()
conn

insert.bind = engine
str(insert)

cur = conn.execute(insert)

cur.inserted_primary_key

2019-07-09 11:46:46,763 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:46:46,764 INFO sqlalchemy.engine.base.Engine ('kim', 'Anonymous, Kim')
2019-07-09 11:46:46,766 INFO sqlalchemy.engine.base.Engine COMMIT


[1]

In [38]:
insert = users.insert()
result = conn.execute(insert, name="lee", fullname="Unknown, Lee")
result.inserted_primary_key

2019-07-09 11:46:48,870 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:46:48,872 INFO sqlalchemy.engine.base.Engine ('lee', 'Unknown, Lee')
2019-07-09 11:46:48,873 INFO sqlalchemy.engine.base.Engine COMMIT


[2]

In [25]:
conn.execute(addresses.insert(), [
    {"user_id":1, "email_adress":"anonymous.kim@test.com"},
    {"user_id":2, "email_adress":"unknown.lee@test.com"}
])

2019-07-09 11:45:53,159 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id) VALUES (?)
2019-07-09 11:45:53,160 INFO sqlalchemy.engine.base.Engine ((1,), (2,))
2019-07-09 11:45:53,161 INFO sqlalchemy.engine.base.Engine ROLLBACK


IntegrityError: (sqlite3.IntegrityError) NOT NULL constraint failed: addresses.email_address
[SQL: INSERT INTO addresses (user_id) VALUES (?)]
[parameters: ((1,), (2,))]
(Background on this error at: http://sqlalche.me/e/gkpj)

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None)

In [40]:
from sqlalchemy.sql import select

query = select([users])
result = conn.execute(query)

for row in result:
    print(row)
    
result = conn.execute(select([users.c.name, users.c.fullname]))

for row in result:
    print(row)

2019-07-09 13:10:46,898 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 13:10:46,899 INFO sqlalchemy.engine.base.Engine ()
(1, 'kim', 'Anonymous, Kim')
(2, 'lee', 'Unknown, Lee')
2019-07-09 13:10:46,901 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2019-07-09 13:10:46,901 INFO sqlalchemy.engine.base.Engine ()
('kim', 'Anonymous, Kim')
('lee', 'Unknown, Lee')


In [41]:
from sqlalchemy import and_, or_, not_
print(users.c.id == addresses.c.user_id)
print(users.c.id == 1)
print((users.c.id == 1).compile().params)
print(or_(users.c.id == addresses.c.user_id, users.c.id == 1))

users.id = addresses.user_id
users.id = :id_1
{'id_1': 1}
users.id = addresses.user_id OR users.id = :id_1


In [42]:
result = conn.execute(select([users]).where(users.c.id==1))
for row in result:
    print(row)
    
result = conn.execute(select([users, addresses]).where(users.c.id==addresses.c.user_id))
for row in result:
    print(row)

2019-07-09 13:18:31,543 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2019-07-09 13:18:31,544 INFO sqlalchemy.engine.base.Engine (1,)
(1, 'kim', 'Anonymous, Kim')
2019-07-09 13:18:31,546 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2019-07-09 13:18:31,546 INFO sqlalchemy.engine.base.Engine ()


In [43]:
from sqlalchemy import join
print(users.join(addresses))

print(users.join(addresses, users.c.id == addresses.c.user_id))

users JOIN addresses ON users.id = addresses.user_id
users JOIN addresses ON users.id = addresses.user_id


In [44]:
query = select([users.c.id, users.c.fullname, addresses.c.email_address]).select_from(users.join(addresses))

result = conn.execute(query).fetchall()
for row in result:
    print(row)

2019-07-09 13:26:04,091 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.fullname, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-07-09 13:26:04,092 INFO sqlalchemy.engine.base.Engine ()


In [45]:
metadata.clear()

In [47]:
metadata.tables

immutabledict({})

------------------------------------------------------

In [49]:
artist = Table('Artist', metadata,
        Column('id', Integer, primary_key=True),
        Column('name', String, nullable=False),
        extend_existing=True)

album = Table('Album', metadata,
        Column('id', Integer, primary_key=True),
        Column('title', String, nullable=False),
        Column('artist_id', Integer, ForeignKey("Artist.id")),
        extend_existing=True)

genre = Table('Genre', metadata,
        Column('id', Integer, primary_key=True),
        Column('name', String, nullable=False),
        extend_existing=True)

track = Table('Track', metadata,
        Column('id', Integer, primary_key=True),
        Column('title', String, nullable=False),
        Column('length', Integer, nullable=False),
        Column('rating', Integer, nullable=False),
        Column('count', Integer, nullable=False),
        Column('album_id', Integer, ForeignKey("Album.id")),
        Column('genre_id', Integer, ForeignKey("Genre.id")),
        extend_existing=True)

metadata.create_all(engine)

2019-07-09 13:39:10,555 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2019-07-09 13:39:10,556 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:10,557 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2019-07-09 13:39:10,558 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:10,559 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2019-07-09 13:39:10,560 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:10,561 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2019-07-09 13:39:10,563 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:10,565 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Artist" (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)


2019-07-09 13:39:10,565 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:39:10,567 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 13:39:10,568 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Genre" (
	id INTEGER NOT NULL, 
	name VARCHAR NOT N

In [53]:
tables = metadata.tables
for table in tables:
    print(table)
    
for table in engine.table_names():
    print(table)

Artist
Album
Genre
Track
2019-07-09 14:07:13,122 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 14:07:13,123 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track
addresses
users


In [52]:
conn.execute(artist.insert(), [
    {"name":"Led Zepplin"},
    {"name":"AC/DC"}
])

conn.execute(album.insert(), [
    {"title":"IV", "artist_id":1},
    {"title":"Who Made Who", "artist_id":2}
])

conn.execute(genre.insert(), [
    {'name':'Rock'},
    {'name':'Metal'}
])

conn.execute(track.insert(), [
    {"title":"Black Dog", "rating":5, "length":297, "count":0, "album_id":1, "genre_id":1},
    {"title":"Stairway", "rating":5, "length":482, "count":0, "album_id":1, "genre_id":1},
    {"title":"About to rock", "rating":5, "length":313, "count":0, "album_id":2, "genre_id":2},
    {"title":"Who Made Who", "rating":5, "length":297, "count":0, "album_id":2, "genre_id":2}
])

2019-07-09 13:49:42,876 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2019-07-09 13:49:42,877 INFO sqlalchemy.engine.base.Engine (('Led Zepplin',), ('AC/DC',))
2019-07-09 13:49:42,879 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 13:49:42,880 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2019-07-09 13:49:42,881 INFO sqlalchemy.engine.base.Engine (('IV', 1), ('Who Made Who', 2))
2019-07-09 13:49:42,883 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 13:49:42,885 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2019-07-09 13:49:42,885 INFO sqlalchemy.engine.base.Engine (('Rock',), ('Metal',))
2019-07-09 13:49:42,887 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 13:49:42,888 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2019-07-09 13:49:42,889 INFO sqlalchemy.engine.base.Engine (('Black Dog', 

In [54]:
print(track.join(album))
print(track.join(album).join(genre))
print(track.join(album).join(artist))
print(track.join(album).join(genre).join(artist))

result = conn.execute(select([track.c.title, album.c.title, genre.c.name, artist.c.name])
                     .select_from(track.join(album).join(genre).join(artist)))

for row in result.fetchall():
    print(row)
    
result = conn.execute(track
                     .select()
                     .select_from(track.join(album).join(genre).join(artist))
                     .where(
                         and_(
                             genre.c.id==1,
                             artist.c.id==1,
                         )
                     )
                    )

for row in result.fetchall():
    print(row)

"Track" JOIN "Album" ON "Album".id = "Track".album_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Artist" ON "Artist".id = "Album".artist_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id
2019-07-09 14:17:10,859 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id
2019-07-09 14:17:10,860 INFO sqlalchemy.engine.base.Engine ()
('Black Dog', 'IV', 'Rock', 'Led Zepplin')
('Stairway', 'IV', 'Rock', 'Led Zepplin')
('About to rock', 'Who Made Who', 'Metal', 'AC/DC')
('Who Made Who', 'Who Made Who', 'Metal', 'AC/DC')
2019-07-09 14:17:10,863 INFO sqlalchemy.engine.base.Engine SELECT

In [55]:
conn.close()
metadata.clear()

In [56]:
metadata.tables

immutabledict({})

====================================================

In [58]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:", echo=True)

In [59]:
from sqlalchemy.ext.declarative import declarative_base

base = declarative_base()

In [61]:
class Artist(base):
    __tablename__ = "Artist"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
class Album(base):
    __tablename__ = "Album"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("Artist.id"))
    
class Genre(base):
    __tablename__ = "Genre"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
class Track(base):
    __tablename__ = "Track"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    length = Column(Integer)
    rating = Column(Integer)
    count = Column(Integer)
    album_id = Column(Integer, ForeignKey("Album.id"))
    genre_id = Column(Integer, ForeignKey("Genre.id"))

In [63]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind = engine)
session = Session()

In [65]:
artist1 = Artist(name="Led Zepplin")
artist2 = Artist(name="AC/DC")

session.add_all([artist1, artist2])
session.commit()

album = [Album(title="IV", artist_id=artist1.id),
         Album(title="Who Made Who", artist_id=artist2.id)]

session.add_all(album)
session.commit()

session.add_all([Genre(name="Rock"), Genre(name="Metal")])
session.commit()

album1 = session.query(Album).filter(Album.artist_id==artist1.id).one()
album2 = session.query(Album).filter(Album.artist_id==artist2.id).one()
genre1 = session.query(Genre).filter(Genre.name=="Rock").filter(Genre.id==1).one()
genre2 = session.query(Genre).filter(Genre.name=="Metal").filter(Genre.id==2).one()
track = [Track(title="Black Dog", rating=5, length=297, count=0, album_id=album1.id, genre_id=genre1.id),
         Track(title="Stairway", rating=5, length=482, count=0, album_id=album1.id, genre_id=genre2.id),
         Track(title="About to rock", rating=5, length=313, count=0, album_id=album2.id, genre_id=genre1.id),
         Track(title="Who Made Who", rating=5, length=297, count=0, album_id=album2.id, genre_id=genre2.id)]

session.add_all(track)
session.commit()

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.OperationalError) no such table: Artist
[SQL: INSERT INTO "Artist" (name) VALUES (?)]
[parameters: ('Led Zepplin',)]
(Background on this error at: http://sqlalche.me/e/e3q8)